In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle5 as pickle # for reading pickle with a more complex way in kaggle
import plotly.express as px # importing the plotly library for mapping
import plotly.graph_objects as go # Graphical projects of the plotly library


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmax-tmin-precipdatayearly201cities/city_info/city_info.csv
/kaggle/input/tmax-tmin-precipdatayearly201cities/compiled_data_on_pkl/compiled_data.pkl


## Reading data 

In [10]:
with open('/kaggle/input/tmax-tmin-precipdatayearly201cities/compiled_data_on_pkl/compiled_data.pkl', "rb") as fh:
    compiled_data = pickle.load(fh)
compiled_data = compiled_data.drop('Unnamed: 0',axis=1)

In [11]:
cities_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')
city_info = pd.read_csv('/kaggle/input/tmax-tmin-precipdatayearly201cities/city_info/city_info.csv')

## showing general data to be plotted

In [12]:
compiled_data.head()

,ID,Date,tmax,tmin,prcp,Name,Lat,Lon
0,USC00042863,1894-01-01,60.0,41.0,0.00,Reno,39.4839,-119.7711
1,USC00042863,1894-01-02,58.0,50.0,0.40,Reno,39.4839,-119.7711
2,USC00042863,1894-01-03,57.0,42.0,0.00,Reno,39.4839,-119.7711
3,USC00042863,1894-01-04,53.0,42.0,0.28,Reno,39.4839,-119.7711
4,USC00042863,1894-01-05,50.0,38.0,0.00,Reno,39.4839,-119.7711


In [13]:
city_info.head()

,Unnamed: 0,Name,ID,Lat,Lon
0,0,Lander,USW00024021,42.8153,-108.7261
1,1,Lander,USW00024018,42.8153,-108.7261
2,2,Cheyenne,USW00014897,41.1519,-104.8061
3,3,Cheyenne,USW00094973,41.1519,-104.8061
4,4,Wausau,USW00014991,44.9258,-89.6256


In [14]:
cities_data.head()

,name,pop,lat,lon
0,New York,8287238,40.730599,-73.986581
1,Los Angeles,3826423,34.053717,-118.242727
2,Chicago,2705627,41.875555,-87.624421
3,Houston,2129784,29.758938,-95.367697
4,Philadelphia,1539313,39.952335,-75.163789


## mapping and first maps

In [22]:
cities_lat = cities_data['lat']
cities_lon = cities_data['lon']
cities_name = cities_data['name']
cities_pop = cities_data['pop']

fig = go.Figure(go.Scattergeo(lat=cities_lat,lon=cities_lon,text=cities_name))
fig.show()